## Imports

In [12]:
import torch
import torch.nn as nn
from torchsummary import summary
from transformer_v2 import Transformer
from utils.constants import *
from utils.function_utils import *

### Init Tansformer

In [2]:
src_vocab_size = 1024
tgt_vocab_size = 1024

transformer = Transformer(src_vocab_size, 
                          tgt_vocab_size, 
                          MODEL_DIM, 
                          MODEL_N_HEADS, 
                          MODEL_N_LAYERS, 
                          MODEL_FF, 
                          dropout=MODEL_DROPOUT, 
                          max_len=MODEL_MAX_SEQ_LEN)

### Model Summary

In [3]:
def print_model_summary(model, input_size):
    print(summary(model, input_size))

In [23]:

class TransformerWrapper(nn.Module):
    def __init__(self, transformer_model):
        super(TransformerWrapper, self).__init__()
        self.transformer_model = transformer_model

    def forward(self, x):

        batch_size = x.size(0)
        seq_length = x.size(1)
        # Split the input tensor into src and tgt, and pad them to the original length
        print(x.shape)
        src = x[:, :seq_length//2]
        tgt = x[:, seq_length//2:seq_length]
        print(src.shape)
        src = torch.cat((src, torch.zeros(batch_size, seq_length//2, dtype=src.dtype, device=src.device)), dim=1)
        tgt = torch.cat((tgt, torch.zeros(batch_size, seq_length//2, dtype=tgt.dtype, device=tgt.device)), dim=1)

        pad_idx = 0

        # Generate masks and pass the tensors to the transformer_model's forward() method
        src_mask, tgt_mask = generate_masks(src, tgt, pad_idx)
        output = self.transformer_model(src, tgt, src_mask, tgt_mask)
        return output



In [24]:
wrapped_transformer = TransformerWrapper(transformer)
input_shape = (32, 256)
print_model_summary(wrapped_transformer, input_shape)

2
32
torch.Size([2, 32, 256])
torch.Size([2, 16, 256])


RuntimeError: Tensors must have same number of dimensions: got 3 and 2